In [1]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data

In [2]:
tasks = pd.read_csv("/Volumes/data/work/picnichealth/tasks.csv")
tasks_group = pd.read_csv("/Volumes/data/work/picnichealth/task_group.csv")
records = pd.read_csv("/Volumes/data/work/picnichealth/records.csv")
timesheet = pd.read_csv("/Volumes/data/work/picnichealth/timesheet_entries.csv")


# Tasks

In [3]:
tasks.head()

,id,created_at,completed_at,task_type,previous_task_task_id,completer_user_id,patient_user_id,record_id
0,00032409-3e0d-4670-9ba4-94922ef010fa,2019-05-07T02:23:50.721Z,2019-05-07T02:23:50.757Z,FINISH_LABELLING,df6000d7-6ac9-40b4-bff9-b53f84f30ead,NaN,d4a59370-08c0-4516-b77e-bc21ae9bb352,d05e40df-2725-487d-86ba-c3825c47c01c
1,00036503-7dee-421c-9b4f-3fefe1884d5a,2019-05-07T02:27:53.191Z,2019-05-16T19:27:35.351Z,CHECK_RECORD_STATUS,f6713596-39ba-43e7-b722-6330bc137a89,3d1b6796-c0b2-4df3-8c45-c70e7986117d,f463eac3-5acb-402c-ae59-1da3fcc82918,NaN
2,0007a86c-e593-4d31-856a-53694b0df491,2019-05-13T18:03:06.895Z,2019-05-13T18:50:16.846Z,ASSIGN_INBOUND_RECORD,9963dabc-2262-470b-8188-22fcc4daef79,9eccd74f-0885-41c3-be93-f9a826470c48,fe20e53c-79b3-4ab3-9f9b-397f24bacd6e,NaN
3,0008d0c1-7c4a-4e8f-adfe-29015e4dc225,2019-05-13T23:15:42.159Z,2019-05-13T23:15:42.188Z,FINISH_LABELLING,ab26af2e-edc1-4794-a78e-53f210a76a44,NaN,cf073b87-f7cc-4363-9ca2-fa1395d60663,e9cc9946-e863-4e1b-8cc5-5d6d11bc6b53
4,000a5e7d-1c86-4486-a72b-afa830d42b3f,2019-05-13T14:51:47.600Z,2019-05-15T23:38:27.769Z,PROCESS_EXCERPTS,9f422db2-f12f-4bed-9d96-de0b78c7db2e,2554c0ae-a994-4b4a-b7e4-594908dab447,a0e0df3f-f193-4a91-8898-5c4d8079a355,0502ac1f-c322-4e55-9524-37bd6bac8102


In [4]:
tasks = tasks.astype({'created_at': 'datetime64'})
tasks = tasks.astype({'completed_at': 'datetime64'})

In [5]:
tasks.nunique()

id                       19900
created_at               19594
completed_at             11024
task_type                   43
previous_task_task_id    11904
completer_user_id           90
patient_user_id           2228
record_id                 3669
dtype: int64

In [6]:
tasks.task_type.unique()

array(['FINISH_LABELLING', 'CHECK_RECORD_STATUS', 'ASSIGN_INBOUND_RECORD',
       'PROCESS_EXCERPTS', 'ANONYMIZE_DICOM', 'PROCESS_RACE',
       'MESSAGE_USER_FOR_ONBOARDING_INFO',
       'MESSAGE_USER_ABOUT_RECORDS_ADDED', 'REVIEW_PROCESSING_COMMENTS',
       'CORRECT_OCR', 'PREPARE_INFORMATION_FOR_RECORD_REQUEST',
       'QUALITY_CHECK_OTHER_EXCERPTS', 'PROCESS_GLOBAL_LABELLING',
       'CORRECT_EXCERPT_OCR', 'QUALITY_CHECK_LABS', 'SEND_REQUEST',
       'QUALITY_CHECK_SEGMENTATION', 'PROCESS_DICOMS', 'SEGMENT_RECORDS',
       'QUALITY_CHECK_GLOBAL_LABELLING', 'CALL_FOR_FAX_NUMBER',
       'MESSAGE_USER_FOR_ORGANIZATION_SPECIFICS',
       'MESSAGE_USER_FOR_COLLECTION_APPROVAL', 'REVIEW_SIGNATURE',
       'ANSWER_INBOUND_CALL', 'AUDIT_CORRECT_EXCERPT_OCR', 'AD_HOC',
       'REVIEW_INVOICE', 'PAY_INVOICE',
       'MESSAGE_USER_ABOUT_UNAVAILABLE_RECORDS',
       'MESSAGE_USER_FOR_SIGNATURE',
       'MESSAGE_USER_ABOUT_DESTROYED_RECORDS',
       'MESSAGE_USER_ABOUT_CARD_FAILURE', 'SEND_DIC

In [7]:
tasks.isnull().sum()

id                           0
created_at                   0
completed_at              8427
task_type                    0
previous_task_task_id     3359
completer_user_id        10368
patient_user_id            139
record_id                 7541
dtype: int64

In [8]:
tasks["duration"] = tasks["completed_at"]-tasks["created_at"]

In [9]:
tasks["total_sec"] = tasks.duration.dt.total_seconds()
tasks["total_hr"] = tasks["total_sec"]/3600

In [10]:
tasks

,id,created_at,completed_at,task_type,previous_task_task_id,completer_user_id,patient_user_id,record_id,duration,total_sec,total_hr
0,00032409-3e0d-4670-9ba4-94922ef010fa,2019-05-07 02:23:50.721,2019-05-07 02:23:50.757,FINISH_LABELLING,df6000d7-6ac9-40b4-bff9-b53f84f30ead,NaN,d4a59370-08c0-4516-b77e-bc21ae9bb352,d05e40df-2725-487d-86ba-c3825c47c01c,0 days 00:00:00.036000,0.036,0.000010
1,00036503-7dee-421c-9b4f-3fefe1884d5a,2019-05-07 02:27:53.191,2019-05-16 19:27:35.351,CHECK_RECORD_STATUS,f6713596-39ba-43e7-b722-6330bc137a89,3d1b6796-c0b2-4df3-8c45-c70e7986117d,f463eac3-5acb-402c-ae59-1da3fcc82918,NaN,9 days 16:59:42.160000,838782.160,232.995044
2,0007a86c-e593-4d31-856a-53694b0df491,2019-05-13 18:03:06.895,2019-05-13 18:50:16.846,ASSIGN_INBOUND_RECORD,9963dabc-2262-470b-8188-22fcc4daef79,9eccd74f-0885-41c3-be93-f9a826470c48,fe20e53c-79b3-4ab3-9f9b-397f24bacd6e,NaN,0 days 00:47:09.951000,2829.951,0.786098
3,0008d0c1-7c4a-4e8f-adfe-29015e4dc225,2019-05-13 23:15:42.159,2019-05-13 23:15:42.188,FINISH_LABELLING,ab26af2e-edc1-4794-a78e-53f210a76a44,NaN,cf073b87-f7cc-4363-9ca2-fa1395d60663,e9cc9946-e863-4e1b-8cc5-5d6d11bc6b53,0 days 00:00:00.029000,0.029,0.000008
4,000a5e7d-1c86-4486-a72b-afa830d42b3f,2019-05-13 14:51:47.600,2019-05-15 23:38:27.769,PROCESS_EXCERPTS,9f422db2-f12f-4bed-9d96-de0b78c7db2e,2554c0ae-a994-4b4a-b7e4-594908dab447,a0e0df3f-f193-4a91-8898-5c4d8079a355,0502ac1f-c322-4e55-9524-37bd6bac8102,2 days 08:46:40.169000,204400.169,56.777825
...,...,...,...,...,...,...,...,...,...,...,...
19895,ffe45191-ceca-47a8-a782-95d2a0a42159,2019-05-08 01:20:51.401,2019-05-08 13:29:50.587,CORRECT_EXCERPT_OCR,0032ab9e-d63e-465b-aee8-ea8a58d40fe8,b9db89ee-fcf4-45de-991a-a16b26fc2f2e,f90e2fba-82aa-46ec-b4a0-ba852d47821c,8e3bd5d2-97c7-42ac-ba05-e68732878c19,0 days 12:08:59.186000,43739.186,12.149774
19896,ffe83d09-54bc-491c-b991-b077c2007f49,2019-05-07 19:08:47.770,2019-05-07 19:08:57.198,SEND_REQUEST,0b2c2078-fe82-4287-9a56-b10fcbd78cb3,15f9a3a9-63c0-45f7-a05a-d7d6cc98c02c,4dd04557-6422-4133-a94b-b20e20543ee9,NaN,0 days 00:00:09.428000,9.428,0.002619
19897,fff00f3f-f898-4f0e-abe4-5553e9465a85,2019-05-11 06:00:40.670,NaT,QUALITY_CHECK_LABS,677c0b33-a400-42cb-afd9-8d7d793307a2,NaN,6d59b718-fbf7-4658-8231-9bfee41b7cbe,510014b7-0ac2-4282-9593-c8cab9c5a278,NaT,NaN,NaN
19898,fff14e4f-d5a6-4d76-a6bb-fa38b24a2bb8,2019-05-14 00:14:46.827,NaT,QUALITY_CHECK_SEGMENTATION,0c96ac77-07e7-4507-b6ba-b35dd4b60838,NaN,ae642296-e078-4ab7-b349-99352e2037ff,aed0a425-0f0b-4b14-9e97-07afa24f9d56,NaT,NaN,NaN


In [11]:
grouped = tasks.groupby('task_type')['total_hr']

ave = grouped.apply(lambda x: np.mean(x)).reset_index()
std = grouped.apply(lambda x: np.std(x)).reset_index()
maxim = grouped.apply(lambda x: np.max(x)).reset_index()
minim = grouped.apply(lambda x: np.min(x)).reset_index()

In [12]:
ave.columns =['task_type', 'ave_duration'] 
std.columns =['task_type', 'std_duration'] 
maxim.columns =['task_type', 'max_duration'] 
minim.columns =['task_type', 'min_duration'] 


In [13]:
task_counts = tasks.value_counts(["task_type"]).reset_index()
task_counts.columns =['task_type', 'count'] 

In [14]:
ave_df = pd.DataFrame(ave)
std_df = pd.DataFrame(std)
max_df = pd.DataFrame(maxim)
min_df = pd.DataFrame(minim)
task_counts_df = pd.DataFrame(task_counts)

In [15]:
max_df = max_df.set_index("task_type")
min_df = min_df.set_index("task_type")
ave_df = ave_df.set_index("task_type")
std_df = std_df.set_index("task_type")
tasks_group = tasks_group.set_index("task_type")
task_counts_df = task_counts_df.set_index("task_type")

In [17]:
task_stat = max_df.join(list([min_df,ave_df,std_df,task_counts_df]))
task_stat["weighted_ave"] = (task_stat["ave_duration"]*task_stat["count"])
task_stat["log_weighted_ave"] = np.log2(task_stat["weighted_ave"]+1) 
task_stat.sort_values(["log_weighted_ave"],ascending=False)

,max_duration,min_duration,ave_duration,std_duration,count,weighted_ave,log_weighted_ave
task_type,,,,,,,
PROCESS_EXCERPTS,275.761232,0.022118,84.787534,54.539336,2891,245120.761961,17.903139
QUALITY_CHECK_SEGMENTATION,261.809441,0.001708,152.568381,72.023035,966,147381.055825,17.169201
QUALITY_CHECK_LABS,262.356393,0.078529,143.408671,66.142139,706,101246.521381,16.627527
CHECK_RECORD_STATUS,318.403790,0.000021,52.491248,61.411845,1817,95376.597495,16.541363
MESSAGE_USER_FOR_ONBOARDING_INFO,168.254089,0.002683,61.515292,55.886329,1452,89320.203288,16.446715
QUALITY_CHECK_OTHER_EXCERPTS,257.776007,0.001133,111.790683,67.689317,740,82725.105209,16.336055
SEGMENT_RECORDS,296.448994,0.004148,77.163035,79.218221,967,74616.654874,16.187229
CORRECT_OCR,128.801748,2.314482,65.558115,63.243633,551,36122.521595,15.140651
REVIEW_PROCESSING_COMMENTS,284.643417,0.014223,39.967863,53.335849,864,34532.233546,15.075698


In [195]:
task_stat = task_stat.reset_index()

In [196]:

tasks_group.columns

Index(['task_group'], dtype='object')

In [197]:
#task_stat = task_stat.join(tasks_group)

task_stat = task_stat.merge(tasks_group,on='task_type',how='left')

In [198]:
task_stat

,task_type,max_duration,min_duration,ave_duration,std_duration,count,weighted_ave,log_weighted_ave,task_group
0,AD_HOC,4411.400000,0.339500,245.539188,794.672483,149,3.658534e+04,15.159017,MISC
1,ANONYMIZE_DICOM,4734.014700,1765.471017,3565.628737,762.924531,394,1.404858e+06,20.421994,MISC
2,ANSWER_INBOUND_CALL,8846.475800,1.105267,630.907931,1910.600464,66,4.163992e+04,15.345714,MISC
3,ASSIGN_INBOUND_RECORD,6880.848067,1.001233,446.169699,918.174708,892,3.979834e+05,18.602352,MISC
4,AUDIT_CORRECT_EXCERPT_OCR,5904.413200,18.169433,1021.867287,1597.139558,44,4.496216e+04,15.456456,OCR
5,CALL_ABOUT_FAX_FAILURE,NaN,NaN,NaN,NaN,9,NaN,NaN,CALL
6,CALL_FOR_FAX_NUMBER,17654.821667,251.219850,3151.937171,3831.237464,62,1.954201e+05,17.576227,CALL
7,CHECK_ANONYMIZED_DICOM,5439.174017,92.926400,3488.863209,1815.341692,31,1.081548e+05,16.722751,CHECK
8,CHECK_RECORD_STATUS,19104.227417,0.001233,3149.474876,3684.710690,1817,5.722596e+06,22.448239,CHECK
9,CONFIRM_RECORDS_RECEIVED,NaN,NaN,NaN,NaN,1,NaN,NaN,MISC


In [244]:
task_stat = task_stat.sort_values(["log_weighted_ave"],ascending=False)
task_stat
task_stat.to_csv("task_stat.csv")

In [238]:
#source = data.wheat()
source = task_stat

bars = alt.Chart(source).mark_bar().encode(
    alt.X('log_weighted_ave:O'),
    alt.Color('task_group:N'),
    alt.Y('task_type:N'),
)


text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=8  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='log_weighted_ave'
)




(bars + text).properties(height=900)

alt.LayerChart(...)

In [245]:
#source = data.wheat()
source = task_stat

bars = alt.Chart(source).mark_bar().encode(
    alt.X('log_weighted_ave:Q'),
    alt.Color('task_group:N'),
    alt.Y('task_type:N'),
)


text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=8  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='log_weighted_ave'
)




(bars + text).properties(height=900)

alt.LayerChart(...)

In [18]:
records.head()

,id,uploaded_at,num_pages
0,b749acd6-5f45-4ce3-be2b-411ac362a67c,2019-05-17T03:03:44.631Z,37
1,4bc36102-de20-4d21-a76e-c3ae96580f3b,2019-04-11T12:40:01.918Z,31
2,b7b82cd1-0d72-43e2-be7e-aec4eea4e4d0,2019-04-04T15:14:38.398Z,12
3,34dbedb4-4ea1-48b7-88ff-ea4fd2b8cfc0,2019-05-03T17:02:36.939Z,25
4,38023f89-1a96-4e9a-9989-337e6cfc7944,2019-04-24T14:35:27.422Z,1


In [248]:
records.columns =['record_id', 'uploaded_at','num_pages'] 
records

,record_id,uploaded_at,num_pages
0,b749acd6-5f45-4ce3-be2b-411ac362a67c,2019-05-17T03:03:44.631Z,37
1,4bc36102-de20-4d21-a76e-c3ae96580f3b,2019-04-11T12:40:01.918Z,31
2,b7b82cd1-0d72-43e2-be7e-aec4eea4e4d0,2019-04-04T15:14:38.398Z,12
3,34dbedb4-4ea1-48b7-88ff-ea4fd2b8cfc0,2019-05-03T17:02:36.939Z,25
4,38023f89-1a96-4e9a-9989-337e6cfc7944,2019-04-24T14:35:27.422Z,1
...,...,...,...
3359,b59c3c95-1f51-4b7b-b786-b056a3561154,2019-05-01T19:37:11.990Z,27
3360,90738f99-e5f2-4361-ad3e-df983395a45a,2019-05-08T14:29:50.904Z,44
3361,61b6a97b-8be9-4ea0-ada0-8cdf9d367616,2019-04-22T20:46:43.884Z,13
3362,36ca384c-17de-4efd-8555-c7fdb0f56536,2019-05-01T19:37:12.944Z,33


In [250]:

tasks.merge(records,on='record_id',how='left')


,id,created_at,completed_at,task_type,previous_task_task_id,completer_user_id,patient_user_id,record_id,duration,total_sec,total_hr,uploaded_at,num_pages
0,00032409-3e0d-4670-9ba4-94922ef010fa,2019-05-07 02:23:50.721,2019-05-07 02:23:50.757,FINISH_LABELLING,df6000d7-6ac9-40b4-bff9-b53f84f30ead,NaN,d4a59370-08c0-4516-b77e-bc21ae9bb352,d05e40df-2725-487d-86ba-c3825c47c01c,0 days 00:00:00.036000,0.036,0.000600,2019-04-11T01:51:30.119Z,2.0
1,00036503-7dee-421c-9b4f-3fefe1884d5a,2019-05-07 02:27:53.191,2019-05-16 19:27:35.351,CHECK_RECORD_STATUS,f6713596-39ba-43e7-b722-6330bc137a89,3d1b6796-c0b2-4df3-8c45-c70e7986117d,f463eac3-5acb-402c-ae59-1da3fcc82918,NaN,9 days 16:59:42.160000,838782.160,13979.702667,NaN,NaN
2,0007a86c-e593-4d31-856a-53694b0df491,2019-05-13 18:03:06.895,2019-05-13 18:50:16.846,ASSIGN_INBOUND_RECORD,9963dabc-2262-470b-8188-22fcc4daef79,9eccd74f-0885-41c3-be93-f9a826470c48,fe20e53c-79b3-4ab3-9f9b-397f24bacd6e,NaN,0 days 00:47:09.951000,2829.951,47.165850,NaN,NaN
3,0008d0c1-7c4a-4e8f-adfe-29015e4dc225,2019-05-13 23:15:42.159,2019-05-13 23:15:42.188,FINISH_LABELLING,ab26af2e-edc1-4794-a78e-53f210a76a44,NaN,cf073b87-f7cc-4363-9ca2-fa1395d60663,e9cc9946-e863-4e1b-8cc5-5d6d11bc6b53,0 days 00:00:00.029000,0.029,0.000483,2019-04-18T21:02:19.874Z,53.0
4,000a5e7d-1c86-4486-a72b-afa830d42b3f,2019-05-13 14:51:47.600,2019-05-15 23:38:27.769,PROCESS_EXCERPTS,9f422db2-f12f-4bed-9d96-de0b78c7db2e,2554c0ae-a994-4b4a-b7e4-594908dab447,a0e0df3f-f193-4a91-8898-5c4d8079a355,0502ac1f-c322-4e55-9524-37bd6bac8102,2 days 08:46:40.169000,204400.169,3406.669483,2019-04-25T21:43:44.708Z,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19895,ffe45191-ceca-47a8-a782-95d2a0a42159,2019-05-08 01:20:51.401,2019-05-08 13:29:50.587,CORRECT_EXCERPT_OCR,0032ab9e-d63e-465b-aee8-ea8a58d40fe8,b9db89ee-fcf4-45de-991a-a16b26fc2f2e,f90e2fba-82aa-46ec-b4a0-ba852d47821c,8e3bd5d2-97c7-42ac-ba05-e68732878c19,0 days 12:08:59.186000,43739.186,728.986433,2019-04-22T21:12:11.418Z,36.0
19896,ffe83d09-54bc-491c-b991-b077c2007f49,2019-05-07 19:08:47.770,2019-05-07 19:08:57.198,SEND_REQUEST,0b2c2078-fe82-4287-9a56-b10fcbd78cb3,15f9a3a9-63c0-45f7-a05a-d7d6cc98c02c,4dd04557-6422-4133-a94b-b20e20543ee9,NaN,0 days 00:00:09.428000,9.428,0.157133,NaN,NaN
19897,fff00f3f-f898-4f0e-abe4-5553e9465a85,2019-05-11 06:00:40.670,NaT,QUALITY_CHECK_LABS,677c0b33-a400-42cb-afd9-8d7d793307a2,NaN,6d59b718-fbf7-4658-8231-9bfee41b7cbe,510014b7-0ac2-4282-9593-c8cab9c5a278,NaT,NaN,NaN,2019-04-17T21:18:54.202Z,66.0
19898,fff14e4f-d5a6-4d76-a6bb-fa38b24a2bb8,2019-05-14 00:14:46.827,NaT,QUALITY_CHECK_SEGMENTATION,0c96ac77-07e7-4507-b6ba-b35dd4b60838,NaN,ae642296-e078-4ab7-b349-99352e2037ff,aed0a425-0f0b-4b14-9e97-07afa24f9d56,NaT,NaN,NaN,2019-05-01T19:43:30.539Z,36.0


In [19]:
records.nunique()

id             3364
uploaded_at    3276
num_pages        70
dtype: int64

In [20]:
records.isnull().sum()

id             0
uploaded_at    0
num_pages      0
dtype: int64

In [21]:
timesheet.head()

,id,start,end,task_id,employee_user_id,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,bad796ee-c8db-4ce9-8dc0-b65e8ae73c8a,2019-05-10T00:13:19.453Z,2019-05-10T00:19:39.688Z,e58419a9-1bb4-4ddb-b32b-d39cb7978320,26667196-e222-46f3-b72a-6c6d5b9a7555,NaN,NaN,NaN
1,3013bfa4-cfa1-471b-a6dc-ae64e8603342,2019-05-11T07:03:09.488Z,2019-05-11T07:15:46.549Z,bb291c71-0c9d-4040-9a14-df239581de69,e7219406-64f8-4413-998d-40c0712fbb06,NaN,NaN,NaN
2,d864fafb-5e27-4c71-ae1b-957acb2aed4c,2019-05-16T14:14:56.899Z,2019-05-16T15:07:05.393Z,1e4f66f7-37b9-4f8a-abc1-0355e95a1021,76f27cdc-e137-4ec2-a287-ae8d5ea5b4ea,NaN,NaN,NaN
3,d66a94e0-2bbf-4f8c-9af8-96d70533068a,2019-05-07T13:08:17.434Z,2019-05-07T13:11:29.743Z,375aa817-5b83-405f-8b7a-aed909dc4bae,e6aef90b-3edb-424a-a93d-f9ce9f559969,NaN,NaN,NaN
4,5bbb69a1-d70e-45f3-853a-2bd100a0487e,2019-05-15T23:39:30.654Z,2019-05-15T23:39:46.603Z,1b6982c6-b3a5-45cd-b4d9-a1802a22fadd,0930c1f5-a41c-4152-8b36-948c6fdee9ff,NaN,NaN,NaN


In [22]:
timesheet.nunique()

id                  11897
start               11897
end                 11835
task_id              7294
employee_user_id       87
Unnamed: 5              0
Unnamed: 6              0
Unnamed: 7              0
dtype: int64

In [23]:
timesheet.isnull().sum()

id                      0
start                   0
end                    35
task_id                 0
employee_user_id        0
Unnamed: 5          11897
Unnamed: 6          11897
Unnamed: 7          11897
dtype: int64

In [24]:
timesheet.head()

,id,start,end,task_id,employee_user_id,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,bad796ee-c8db-4ce9-8dc0-b65e8ae73c8a,2019-05-10T00:13:19.453Z,2019-05-10T00:19:39.688Z,e58419a9-1bb4-4ddb-b32b-d39cb7978320,26667196-e222-46f3-b72a-6c6d5b9a7555,NaN,NaN,NaN
1,3013bfa4-cfa1-471b-a6dc-ae64e8603342,2019-05-11T07:03:09.488Z,2019-05-11T07:15:46.549Z,bb291c71-0c9d-4040-9a14-df239581de69,e7219406-64f8-4413-998d-40c0712fbb06,NaN,NaN,NaN
2,d864fafb-5e27-4c71-ae1b-957acb2aed4c,2019-05-16T14:14:56.899Z,2019-05-16T15:07:05.393Z,1e4f66f7-37b9-4f8a-abc1-0355e95a1021,76f27cdc-e137-4ec2-a287-ae8d5ea5b4ea,NaN,NaN,NaN
3,d66a94e0-2bbf-4f8c-9af8-96d70533068a,2019-05-07T13:08:17.434Z,2019-05-07T13:11:29.743Z,375aa817-5b83-405f-8b7a-aed909dc4bae,e6aef90b-3edb-424a-a93d-f9ce9f559969,NaN,NaN,NaN
4,5bbb69a1-d70e-45f3-853a-2bd100a0487e,2019-05-15T23:39:30.654Z,2019-05-15T23:39:46.603Z,1b6982c6-b3a5-45cd-b4d9-a1802a22fadd,0930c1f5-a41c-4152-8b36-948c6fdee9ff,NaN,NaN,NaN


In [25]:
timesheet.drop(timesheet.columns[[5, 6, 7]], axis = 1, inplace = True) 
timesheet.head()

,id,start,end,task_id,employee_user_id
0,bad796ee-c8db-4ce9-8dc0-b65e8ae73c8a,2019-05-10T00:13:19.453Z,2019-05-10T00:19:39.688Z,e58419a9-1bb4-4ddb-b32b-d39cb7978320,26667196-e222-46f3-b72a-6c6d5b9a7555
1,3013bfa4-cfa1-471b-a6dc-ae64e8603342,2019-05-11T07:03:09.488Z,2019-05-11T07:15:46.549Z,bb291c71-0c9d-4040-9a14-df239581de69,e7219406-64f8-4413-998d-40c0712fbb06
2,d864fafb-5e27-4c71-ae1b-957acb2aed4c,2019-05-16T14:14:56.899Z,2019-05-16T15:07:05.393Z,1e4f66f7-37b9-4f8a-abc1-0355e95a1021,76f27cdc-e137-4ec2-a287-ae8d5ea5b4ea
3,d66a94e0-2bbf-4f8c-9af8-96d70533068a,2019-05-07T13:08:17.434Z,2019-05-07T13:11:29.743Z,375aa817-5b83-405f-8b7a-aed909dc4bae,e6aef90b-3edb-424a-a93d-f9ce9f559969
4,5bbb69a1-d70e-45f3-853a-2bd100a0487e,2019-05-15T23:39:30.654Z,2019-05-15T23:39:46.603Z,1b6982c6-b3a5-45cd-b4d9-a1802a22fadd,0930c1f5-a41c-4152-8b36-948c6fdee9ff
